# Latent Dirichlet Allocation

<img src='https://miro.medium.com/max/1014/1*2uj6t3gNv76SpHrWf5-z-A.jpeg' img/>

<img src='https://miro.medium.com/max/2361/1*fCc0JT3W-1ViYyw0hJ7rdA.jpeg' img/>

### Resources: 

https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-latent-dirichlet-allocation-437c81220158

https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d

https://www.coursera.org/lecture/text-mining/3-9-latent-dirichlet-allocation-lda-part-1-deiXc

https://www.coursera.org/lecture/ml-clustering-and-retrieval/mixed-membership-models-for-documents-hQBJI

## Data

We will be using articles from NPR (National Public Radio), obtained from their website [www.npr.org](http://www.npr.org)

In [1]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 973kB/s eta 0:00:01
     |████████████████████████████████| 552kB 1.6MB/s eta 0:00:01
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=918c918f8a4e773c9874f1f4afb96837cfe482de6506f4c369a5b26805137440
  Stored in directory: /Users/chrisshaw/Library/Caches/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32040 sha256=d6e23f13e56f8cfef9704894dc6bfd1a1a4e112fc37ef029c8e806a7b6e612b6
  Stored in directory: /Users/chrisshaw/Library/Caches/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [10]:
import pandas as pd

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
npr = pd.read_csv('npr.csv')

In [13]:
npr.shape

(11992, 1)

In [14]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

## Preprocessing

In [15]:
import re 
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def spacy_tokenizer(text):
    # remove html tags from all of the text before processing
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    # Creating our token object, which is used to create documents with linguistic annotations.
    # we disabled the parser and ner parts of the pipeline in order to speed up parsing
    mytokens = nlp(cleantext, disable=['parser', 'ner'])

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [17]:
cv = CountVectorizer(tokenizer=spacy_tokenizer, max_df=0.90, min_df=10, stop_words='english')

In [18]:
dtm = cv.fit_transform(npr['Article'])

In [19]:
dtm

<11992x17335 sparse matrix of type '<class 'numpy.int64'>'
	with 2635570 stored elements in Compressed Sparse Row format>

## LDA

In [20]:
from sklearn.decomposition import LatentDirichletAllocation

In [23]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=20,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=20,
                          mean_change_tol=0.001, n_components=20, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [24]:
# This can take awhile, we're dealing with a large amount of documents!

lda_output = lda_model.fit_transform(dtm)


### Diagnose model performance with perplexity and log-likelihood
A model with higher log-likelihood and lower perplexity (exp(-1. * log-likelihood per word)) is considered to be good. Let’s check for our model.



In [25]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(dtm))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(dtm))

# See model parameters
print(lda_model.get_params())

Log Likelihood:  -33400222.4970819
Perplexity:  2551.123667956309
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'online', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 20, 'mean_change_tol': 0.001, 'n_components': 20, 'n_jobs': -1, 'perp_tol': 0.1, 'random_state': 100, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


## Showing Stored Words

In [26]:
len(cv.get_feature_names())

17335

In [27]:
import random

In [36]:
for i in range(10):
    random_word_id = random.randint(0,17335)
    print(cv.get_feature_names()[random_word_id])

nascent
getty
correspondent
santo
motive
dianne
selling
attorney
superdelegate
latent


In [37]:
for i in range(10):
    random_word_id = random.randint(0,17335)
    print(cv.get_feature_names()[random_word_id])

regulation
bishop
regina
tremendous
daughter
fashioned
benchmark
regulatory
technical
lauren


### Showing Top Words Per Topic

In [30]:
len(lda_model.components_)

20

In [31]:
lda_model.components_

array([[2.95263042e+02, 7.25119798e+00, 5.00000236e-02, ...,
        2.66060851e+03, 5.00000012e-02, 2.91460463e+00],
       [3.36396661e+01, 4.02955070e+00, 2.15230304e+02, ...,
        3.69736074e+03, 5.00000008e-02, 1.06743824e+01],
       [1.28369171e+02, 5.00000038e-02, 5.00000038e-02, ...,
        2.77567185e+03, 2.84902031e+00, 4.50064874e+00],
       ...,
       [1.64902804e+03, 8.32752209e+01, 2.73369892e+01, ...,
        9.99917571e+03, 4.12498825e+01, 5.00000040e-02],
       [5.00005188e-02, 5.00000015e-02, 5.00000008e-02, ...,
        1.41526963e+03, 5.00000019e-02, 5.00000011e-02],
       [7.21016893e+02, 5.00000019e-02, 5.00000033e-02, ...,
        2.88533237e+03, 1.03495770e+01, 5.00000070e-02]])

In [32]:
len(lda_model.components_[0])

17335

In [33]:
single_topic = lda_model.components_[0]

In [34]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([ 1549,  2084, 11552, ...,  7639, 12189, 16014])

In [35]:
# Word least representative of this topic
single_topic[18302]

IndexError: index 18302 is out of bounds for axis 0 with size 17335

In [ ]:
# Word most representative of this topic
single_topic[42993]

In [ ]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

In [ ]:
top_word_indices = single_topic.argsort()[-10:]

In [ ]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [ ]:
for index,topic in enumerate(lda_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

### How to see the dominant topic in each document?
To classify a document as belonging to a particular topic, a logical approach is to see which topic has the highest contribution to that document and assign it.

In [ ]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(dtm)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_topics)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

### Review topics distribution across documents

In [ ]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, dtm, vectorizer, mds='tsne')
panel

### Get the top 15 keywords each topic

In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

### Attaching Discovered Topic Labels to Original Articles

In [ ]:
dtm

In [ ]:
dtm.shape

In [ ]:
len(npr)

In [ ]:
topic_results = lda_model.transform(dtm)

In [ ]:
topic_results.shape

In [ ]:
topic_results[0]

In [ ]:
topic_results[0].round(2)

In [ ]:
topic_results[0].argmax()

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [ ]:
npr.head()

In [ ]:
topic_results.argmax(axis=1)

In [ ]:
npr['Topic'] = topic_results.argmax(axis=1)

In [ ]:
npr.head(10)

## Great work!

### How to GridSearch the best LDA model?
The most important tuning parameter for LDA models is n_components (number of topics). In addition, I am going to search learning_decay (which controls the learning rate) as well.

Besides these, other possible search params could be learning_offset (downweigh early iterations. Should be > 1) and max_iter. These could be worth experimenting if you have enough computing resources.

Be warned, the grid search constructs multiple LDA models for all possible combinations of param values in the param_grid dict. So, this process can consume a lot of time and resources.

In [ ]:
from sklearn.model_selection import GridSearchCV


# Define Search Param
search_params = {'n_components': [15, 20, 25],'learning_decay': [.5, .7]}

# Init the Model
lda = LatentDirichletAllocation(max_iter=25)

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params, cv=3, verbose=2, n_jobs = -1)

# Do the Grid Search
model.fit(dtm)

How to see the best topic model and its parameters?


In [ ]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))

### Compare LDA Model Performance Scores

In [ ]:
# Get Log Likelyhoods from Grid Search Output
n_topics = [15, 20, 25]
log_likelyhoods_5 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.5]
log_likelyhoods_7 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.7]

# Show graph
plt.figure(figsize=(12, 8))
plt.plot(n_topics, log_likelyhoods_5, label='0.5')
plt.plot(n_topics, log_likelyhoods_7, label='0.7')
plt.title("Choosing Optimal LDA Model")
plt.xlabel("Num Topics")
plt.ylabel("Log Likelyhood Scores")
plt.legend(title='Learning decay', loc='best')
plt.show()

https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/